In [1]:
import numpy as np
import cv2
import mediapipe as mp
import time
import psutil

from tensorflow.keras.models import load_model

2024-04-23 09:43:00.934990: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 09:43:00.979026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 09:43:00.979058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 09:43:00.980151: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 09:43:00.986098: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 09:43:00.986524: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

I0000 00:00:1713845583.060933   19130 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713845583.063718   19272 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()

    return pose

In [4]:
SEQUENCE_LENGTH = 40

In [5]:
# model = load_model("../models/LSTM.h5")
model = load_model("../models/LSTM_Attention_128HUs.h5")
# model = load_model("../models/CNN_BLSTM model.h5")


cap = cv2.VideoCapture('../dataset/squat/squat_11.mp4')
# cap = cv2.VideoCapture('../dataset/hammer curl/hammer curl_11.mp4')
# cap = cv2.VideoCapture('../dataset/barbell biceps curl/barbell biceps curl_11.mp4')
# cap = cv2.VideoCapture(0)



sequence = []
actions = np.array(['barbell biceps curl','bench press','chest fly machine','deadlift','decline bench press',
                  'hammer curl','hip thrust','incline bench press','lat pulldown','lateral raise',
                  'leg extension','leg raises','plank','pull Up','push-up','romanian deadlift',
                  'russian twist','shoulder press','squat','t bar row','tricep Pushdown','tricep dips'])
current_action = ''

prev_frame_time = 0
new_frame_time = 0

while cap.isOpened():
      ret, frame = cap.read()
    
      if not ret: 
            break
      
      rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = pose.process(rgb_frame)
    
      if results.pose_landmarks:
            landmarks = extract_keypoints(results)
            mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      else: 
            continue 
      
      sequence.append(landmarks)
      
      if len(sequence) == SEQUENCE_LENGTH:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
            
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
            if (confidence < 0.8):
                  current_action = ''
      
      # CPU analysis
      new_frame_time = time.time() 

      fps = 1/(new_frame_time-prev_frame_time) 
      prev_frame_time = new_frame_time 
      fps = str(int(fps)) 

      ram_usage = psutil.virtual_memory().percent
      cpu_usage = psutil.cpu_percent()

      # putting the FPS count on the frame 
      cv2.putText(frame, 'FPS: {}'.format(fps), (1000, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 255, 0), 2) 

      # Display the RAM usage
      cv2.putText(frame, f"RAM Usage: {ram_usage}%", (1000, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 255, 0), 2)

      # Display CPU usage
      cv2.putText(frame, f"CPU Usage: {cpu_usage}%", (1000, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100,255,0), 2)
      
      cv2.rectangle(frame, (0,0), (360, 40), 0.5, -1)
      cv2.putText(frame, 'Exercise ' + current_action, (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    

      cv2.imshow('Output Video', frame)

      if cv2.waitKey(10) & 0xFF==ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/dell/.local/lib/python3.10/site-packages/cv2/qt/plugins"
